HAYSTACK AI

In [5]:
%pip install haystack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [1]:
domain = "Automated Order System"
question = f"Based on a {domain}'s data, suggest 1 natural language search prompt for every type of queries in the documents to get meaningful insights from the data."


In [2]:
import json
file_path = '/content/AOS1005.json'
with open(file_path, 'r') as file:
    data = json.load(file)

ATTRIBUTE_TYPES = {
    "Continuous": [],
    "Categorical": [],
    "Ordered": [],
    "Boolean": [],
    "Database ID": [],
    "Generic string": [],
    "Timezone": [],
    "Timestamp": [],
}

ENTITY_TYPES = {
    "Fact": [],
    "Dimension": []
}


"""
ATTRIBUTE_TYPES = {
    "Continuous": ["sales", "temperature", "revenue", "age", "growth", "score", "price"],
    "Categorical": ["region", "category", "department", "store"],
    "Ordered": ["rank", "order", "level"],
    "Boolean": ["true", "false"],
    "Database ID": ["id", "identifier", "user_id", "product_id"],
    "Generic string": ["name", "description", "keyword", "phrase"]
}

ENTITY_TYPES = {
    "Fact": ["sales", "temperature", "revenue", "growth", "score", "price"],
    "Dimension": ["region", "category", "department", "store", "user", "product"]
}
"""

def classify_attribute(attr):
    logical_data_type = attr.get("logicalDataType")
    if logical_data_type == "CATEGORICAL":
        return "Categorical"
    elif logical_data_type == "DATABASE_ID":
        return "Database ID"
    elif logical_data_type == "CONTINUOUS":
        return "Continuous"
    elif logical_data_type == "BOOLEAN":
        return "Boolean"
    elif logical_data_type == "GENERIC_STRING":
        return "Generic string"
    elif logical_data_type == "TIMEZONE":
        return "Timezone"
    elif logical_data_type == "TIMESTAMP":
        return "Timestamp"
    else:
        return None

for entity in data.get("entityList", []):
    entity_type = entity.get("entityType")
    entity_name = entity.get("canonicalName")

    if entity_type == "Fact" or entity_type == "Dimension":
        ENTITY_TYPES[entity_type].append(entity_name)

    for attribute in entity.get("attributeList", []):
        attr_type = classify_attribute(attribute)
        if attr_type:
            ATTRIBUTE_TYPES[attr_type].append(attribute.get("canonicalName"))

print("ATTRIBUTE_TYPES =", json.dumps(ATTRIBUTE_TYPES, indent=4))
print("ENTITY_TYPES =", json.dumps(ENTITY_TYPES, indent=4))
#query_ranked = "what are the top k <continuous_attri> "

ATTRIBUTE_TYPES = {
    "Continuous": [
        "sentiment_score",
        "volatility",
        "volumevolatility",
        "localhour",
        "totalnonadherenceminutes",
        "reportminutes",
        "adheringtoschedule",
        "paidminutes",
        "absentminutes",
        "duration_min",
        "shiftminutes",
        "exceptiontminutes",
        "durationoverride",
        "paid",
        "durationhours",
        "duration",
        "number_of_holds",
        "number_of_conferences",
        "number_of_transfers",
        "num_of_assessments",
        "bpuserid",
        "seconds_since_previous_contact",
        "paid",
        "work",
        "durationhours",
        "inum",
        "total_hold_time",
        "duration",
        "seconds_of_silence",
        "num_of_agent_cross",
        "number_of_holds",
        "seconds_of_agent_talk",
        "seconds_of_other_talk",
        "num_of_evaluations",
        "capture_health_score",
        "localhour_minute",
        "ac

In [4]:
ATTRIBUTE_TYPES['Database ID']

['interaction_id',
 'agent_qm_analytics_employee_id',
 'employeeid',
 'qm_analytics_employee_id',
 'personid',
 'organizationid',
 'original_id',
 'category_id',
 'dateid',
 'id',
 'organizationid',
 'employeeid',
 'evaluations_forms_id',
 'form_id',
 'form_revision_id',
 'form_component_id',
 'employeeid',
 'organizationid',
 'employeeid',
 'id',
 'sid',
 'campaignid',
 'organizationid',
 'activityid',
 'id',
 'parentid',
 'tenantid',
 'id',
 'employee_id',
 'application_id',
 'computer_id',
 'employeeid',
 'organizationid',
 'sid',
 'organizationid',
 'id',
 'activityid',
 'workresourceid',
 'organizationdayid',
 'spdayid',
 'shiftassignmentid',
 'tenantid',
 'interaction_id',
 'instance_id',
 'tenantid',
 'employeeid',
 'activityid',
 'organizationdayid',
 'organizationid',
 'spdayid',
 'adherenceexceptionid',
 'id',
 'last_interaction_id',
 'organizationid',
 'employeeid',
 'contactid',
 'id',
 'employeeid',
 'activityid',
 'organizationdayid',
 'organizationid',
 'spdayid',
 'shif

In [4]:
len(ENTITY_TYPES['Fact'])+len(ENTITY_TYPES['Dimension'])

61

In [7]:
import os
from haystack import Pipeline, Document
from haystack.utils import Secret
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder

# Write documents to InMemoryDocumentStore
document_store = InMemoryDocumentStore()
document_store.write_documents([
    Document(content=f"""
1. Time Series Queries
Uses the following(but not limited to) attributes and entity types:
Attribute Type:
Continuous: To measure values over time.
Timezone: To account for time zone differences.
Entity Type:
Fact: To track measurements over time (e.g., sales, temperature).
Key-Value Conditions: isTimeSensitive, datetimeGranularity

Query semantics: "<Action> <Subject> <Time Frame> <Conditions> <Granularity>"
Here, <Subject> must be taken from {ENTITY_TYPES['Fact']}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']} and {ATTRIBUTE_TYPES['Timezone']}.
"""),
    Document(content=f"""
2. Time and Context Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Continuous: For values that change over time.
Categorical: For context (e.g., regions, categories).
Timezone: To standardise time data.
Entity Type:
Fact: For time-based measurements.
Dimension: For contextual information (e.g., region, department).
Key-Value Conditions: isTimeSensitive, datetimeGranularity

Query semantics: <Action> <Subject> <Time Frame> <Context> <Conditions> <Granularity>
Here, <Subject> must be taken from {ENTITY_TYPES}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']}, {ATTRIBUTE_TYPES['Categorical']} and {ATTRIBUTE_TYPES['Timezone']}.
"""),
    Document(content=f"""
3. Growth Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Continuous: To measure growth metrics (e.g., revenue, user base).
Ordered: To rank growth rates.
Entity Type:
Fact: To calculate growth based on factual data over time.
Key-Value Conditions: aggregationMethods, rowCount

Query semantics: <Action> <Subject> <Time Frame> <Comparison> <Conditions>
Here, <Subject> must be taken from {ENTITY_TYPES['Fact']}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']} and {ATTRIBUTE_TYPES['Ordered']}.
"""),
Document(content=f"""
4. Ranked Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Ordered: To rank items (e.g., top sales, highest scores).
Continuous: For ranking based on continuous values.
Entity Type:
Fact: To rank items based on factual data.
Dimension: To provide contextual ranking (e.g., by category).
Key-Value Conditions: rank, sortOrder

Query semantics: <Action> <Subject> <Criteria> <Context> <Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']} and {ATTRIBUTE_TYPES['Ordered']}.
"""),
Document(content=f"""
5. Key Value Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Database ID: To uniquely identify key-value pairs.
Generic string: For key names and values.
Entity Type:
Dimension: For storing key-value pairs.
Key-Value Conditions: primaryKeyAttribute, entityName

Query semantics: <Action> <Subject> <Key Condition> <Value Condition> <Additional Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES['Dimension']}.
Rest should be taken from {ATTRIBUTE_TYPES['Database ID']} and {ATTRIBUTE_TYPES['Generic string']}.
"""),
Document(content=f"""
6. Identification Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Database ID: For unique identification.
Generic string: For names and descriptions.
Entity Type:
Dimension: For identifying entities (e.g., user, product).
Key-Value Conditions: primaryKeyAttribute, entityName

Query semantics: <Action> <Subject> <Identifier> <Additional Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES['Dimension']}.
Rest should be taken from {ATTRIBUTE_TYPES['Database ID']} and {ATTRIBUTE_TYPES['Generic string']}.
"""),
Document(content=f"""
7. Comparison Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Continuous: To compare numerical values.
Categorical: For comparing categories.
Boolean: To compare true/false conditions.
Entity Type:
Fact: For comparing factual data.
Dimension: For contextual comparison.
Key-Value Conditions: context, correlatedAttributes

Query semantics: <Action> <Subject> <Criteria> <Context> <Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']}, {ATTRIBUTE_TYPES['Categorical']} and {ATTRIBUTE_TYPES['Boolean']}.
"""),
Document(content=f"""
8. Exclusion Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Categorical: To exclude certain categories.
Boolean: To exclude true/false conditions.
Entity Type:
Dimension: For excluding specific dimensions (e.g., regions, departments).
Key-Value Conditions: excludedCategories, defaultFilters

Query semantics: <Action> <Subject> <Exclusion Criteria> <Additional Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES['Dimension']}.
Rest should be taken from {ATTRIBUTE_TYPES['Categorical']} and {ATTRIBUTE_TYPES['Boolean']}.
"""),
Document(content=f"""
9. Range Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Continuous: To specify ranges (e.g., price range, date range).
Ordered: To define ordered ranges.
Entity Type:
Fact: For data falling within specific ranges.
Dimension: For contextual ranges (e.g., date range, age range).
Key-Value Conditions: datetimeGranularity, defaultFilters

Query semantics: <Action> <Subject> <Range Criteria> <Context> <Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']} and {ATTRIBUTE_TYPES['Ordered']}.
"""),
Document(content=f"""
10. Computation Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Continuous: For calculations (e.g., sum, average).
Boolean: To include/exclude in computations.
Entity Type:
Fact: To perform computations on factual data.
Key-Value Conditions: aggregationMethods, isComputedAttribute

Query semantics: <Action> <Subject> <Computation Method> <Context> <Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES['Fact']}.
Rest should be taken from {ATTRIBUTE_TYPES['Continuous']} and {ATTRIBUTE_TYPES['Boolean']}.
"""),
Document(content=f"""
11. Text Queries
Uses the following(but not limited to) attribute and entity types:
Attribute Type:
Generic string: For text-based searches.
Searchable/unsearchable: To determine if text attributes can be searched.
Entity Type:
Dimension: For searching within descriptive attributes.
Key-Value Conditions: mostUsedDataValues, customerSynonyms

Query semantics: <Action> <Subject> <Search Criteria> <Context> <Conditions>.
Here, <Subject> must be taken from {ENTITY_TYPES['Dimension']}.
Rest should be taken from {ATTRIBUTE_TYPES['Generic string']}.
"""),
Document(content=f"""
ATTRIBUTE_TYPES = {ATTRIBUTE_TYPES}

ENTITY_TYPES = {ENTITY_TYPES}
""")
])

# Build a RAG pipeline
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)
from google.colab import userdata

llm = OpenAIGenerator(api_key=Secret.from_token(userdata.get('openai-api-key')))

rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")


results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

print(results["llm"]["replies"])

['1. Time Series Query: "Show the trend of sales volume over the past month."\n2. Identification Query: "Retrieve the details of the customer with the ID \'12345\'."\n3. Key Value Query: "Find all interactions with the campaign name \'Winter Sale\'."\n4. Comparison Query: "Compare the performance of employees based on their evaluation scores."\n5. Range Query: "Identify all interactions with a duration between 10-20 minutes."\n6. Time and Context Query: "Show the employee schedules for the current week in the sales department."\n7. Growth Query: "Calculate the growth rate of customer interactions over the past year."\n8. Computation Query: "Calculate the average duration of interactions for each employee."\n9. Ranked Query: "Rank employees based on their adherence to schedules."\n10. Text Query: "Search for interactions related to the \'Customer Support\' campaign."']


In [8]:
res = results["llm"]["replies"]
res[0].split('\n')

['1. Time Series Query: "Show the trend of sales volume over the past month."',
 '2. Identification Query: "Retrieve the details of the customer with the ID \'12345\'."',
 '3. Key Value Query: "Find all interactions with the campaign name \'Winter Sale\'."',
 '4. Comparison Query: "Compare the performance of employees based on their evaluation scores."',
 '5. Range Query: "Identify all interactions with a duration between 10-20 minutes."',
 '6. Time and Context Query: "Show the employee schedules for the current week in the sales department."',
 '7. Growth Query: "Calculate the growth rate of customer interactions over the past year."',
 '8. Computation Query: "Calculate the average duration of interactions for each employee."',
 '9. Ranked Query: "Rank employees based on their adherence to schedules."',
 '10. Text Query: "Search for interactions related to the \'Customer Support\' campaign."']

In [1]:
conditions = """
1. Time Series Queries
Conditions:
Attribute Type:
Continuous: To measure values over time.
Timezone: To account for time zone differences.
Entity Type:
Fact: To track measurements over time (e.g., sales, temperature).
Key-Value Conditions: isTimeSensitive, datetimeGranularity

def is_time_series_query(attribute_type, entity_type):
    return attribute_type == "Continuous" and entity_type == "Fact"

query_1 = "What are the monthly sales for the past year?"
attribute_type_1 = "Continuous"  # Sales over time
entity_type_1 = "Fact"
print(is_time_series_query(attribute_type_1, entity_type_1))  # Output: True


2. Time and Context Queries
Conditions:
Attribute Type:
Continuous: For values that change over time.
Categorical: For context (e.g., regions, categories).
Timezone: To standardise time data.
Entity Type:
Fact: For time-based measurements.
Dimension: For contextual information (e.g., region, department).
Key-Value Conditions: isTimeSensitive, datetimeGranularity

def is_time_and_context_query(attribute_types, entity_types):
    return "Continuous" in attribute_types and "Categorical" in attribute_types and "Fact" in entity_types and "Dimension" in entity_types

query_2 = "What are the monthly sales for each region?"
attribute_types_2 = ["Continuous", "Categorical"]  # Sales over time and by region
entity_types_2 = ["Fact", "Dimension"]
print(is_time_and_context_query(attribute_types_2, entity_types_2))  # Output: True



3. Growth Queries
Conditions:
Attribute Type:
Continuous: To measure growth metrics (e.g., revenue, user base).
Ordered: To rank growth rates.
Entity Type:
Fact: To calculate growth based on factual data over time.
Key-Value Conditions: aggregationMethods, rowCount


def is_growth_query(attribute_types, entity_type):
    return "Continuous" in attribute_types and "Ordered" in attribute_types and entity_type == "Fact"

query_3 = "What is the growth rate of the user base over the last year?"
attribute_types_3 = ["Continuous", "Ordered"]  # User base growth over time
entity_type_3 = "Fact"
print(is_growth_query(attribute_types_3, entity_type_3))  # Output: True


4. Ranked Queries
Conditions:
Attribute Type:
Ordered: To rank items (e.g., top sales, highest scores).
Continuous: For ranking based on continuous values.
Entity Type:
Fact: To rank items based on factual data.
Dimension: To provide contextual ranking (e.g., by category).
Key-Value Conditions: rank, sortOrder

def is_ranked_query(attribute_types, entity_types):
    return "Ordered" in attribute_types and "Continuous" in attribute_types and ("Fact" in entity_types or "Dimension" in entity_types)

query_4 = "What are the top 10 highest sales regions?"
attribute_types_4 = ["Ordered", "Continuous"]  # Ranking sales
entity_types_4 = ["Fact", "Dimension"]
print(is_ranked_query(attribute_types_4, entity_types_4))  # Output: True

5. Key Value Queries
Conditions:
Attribute Type:
Database ID: To uniquely identify key-value pairs.
Generic string: For key names and values.
Entity Type:
Dimension: For storing key-value pairs.
Key-Value Conditions: primaryKeyAttribute, entityName

def is_key_value_query(attribute_types, entity_type):
    return "Database ID" in attribute_types and "Generic string" in attribute_types and entity_type == "Dimension"

query_5 = "What is the value of the setting for user_id 123?"
attribute_types_5 = ["Database ID", "Generic string"]  # Key-value pair
entity_type_5 = "Dimension"
print(is_key_value_query(attribute_types_5, entity_type_5))  # Output: True

6. Identification Queries
Conditions:
Attribute Type:
Database ID: For unique identification.
Generic string: For names and descriptions.
Entity Type:
Dimension: For identifying entities (e.g., user, product).
Key-Value Conditions: primaryKeyAttribute, entityName

def is_identification_query(attribute_types, entity_type):
    return "Database ID" in attribute_types and "Generic string" in attribute_types and entity_type == "Dimension"

query_6 = "Identify the product with ID 456."
attribute_types_6 = ["Database ID", "Generic string"]  # Identification
entity_type_6 = "Dimension"
print(is_identification_query(attribute_types_6, entity_type_6))  # Output: True

7. Comparison Queries
Conditions:
Attribute Type:
Continuous: To compare numerical values.
Categorical: For comparing categories.
Boolean: To compare true/false conditions.
Entity Type:
Fact: For comparing factual data.
Dimension: For contextual comparison.
Key-Value Conditions: context, correlatedAttributes

def is_comparison_query(attribute_types, entity_types):
    return any(attr in attribute_types for attr in ["Continuous", "Categorical", "Boolean"]) and ("Fact" in entity_types or "Dimension" in entity_types)


query_7 = "Compare the sales figures between Q1 and Q2."
attribute_types_7 = ["Continuous", "Categorical"]  # Comparing sales figures
entity_types_7 = ["Fact", "Dimension"]
print(is_comparison_query(attribute_types_7, entity_types_7))  # Output: True

8. Exclusion Queries
Conditions:
Attribute Type:
Categorical: To exclude certain categories.
Boolean: To exclude true/false conditions.
Entity Type:
Dimension: For excluding specific dimensions (e.g., regions, departments).
Key-Value Conditions: excludedCategories, defaultFilters

def is_exclusion_query(attribute_types, entity_type):
    return any(attr in attribute_types for attr in ["Categorical", "Boolean"]) and entity_type == "Dimension"

query_8 = "Show sales figures excluding the electronics category."
attribute_types_8 = ["Categorical", "Boolean"]  # Excluding a category
entity_type_8 = "Dimension"
print(is_exclusion_query(attribute_types_8, entity_type_8))  # Output: True

9. Range Queries
Conditions:
Attribute Type:
Continuous: To specify ranges (e.g., price range, date range).
Ordered: To define ordered ranges.
Entity Type:
Fact: For data falling within specific ranges.
Dimension: For contextual ranges (e.g., date range, age range).
Key-Value Conditions: datetimeGranularity, defaultFilters

def is_range_query(attribute_types, entity_types):
    return any(attr in attribute_types for attr in ["Continuous", "Ordered"]) and ("Fact" in entity_types or "Dimension" in entity_types)

query_9 = "What are the sales figures between January and March?"
attribute_types_9 = ["Continuous", "Ordered"]  # Sales in a date range
entity_types_9 = ["Fact", "Dimension"]
print(is_range_query(attribute_types_9, entity_types_9))  # Output: True


10. Computation Queries
Conditions:
Attribute Type:
Continuous: For calculations (e.g., sum, average).
Boolean: To include/exclude in computations.
Entity Type:
Fact: To perform computations on factual data.
Key-Value Conditions: aggregationMethods, isComputedAttribute

def is_computation_query(attribute_types, entity_type):
    return any(attr in attribute_types for attr in ["Continuous", "Boolean"]) and entity_type == "Fact"

query_10 = "Calculate the average sales for the past year."
attribute_types_10 = ["Continuous", "Boolean"]  # Average sales calculation
entity_type_10 = "Fact"
print(is_computation_query(attribute_types_10, entity_type_10))  # Output: True

11. Text Queries
Conditions:
Attribute Type:
Generic string: For text-based searches.
Searchable/unsearchable: To determine if text attributes can be searched.
Entity Type:
Dimension: For searching within descriptive attributes.
Key-Value Conditions: mostUsedDataValues, customerSynonyms

def is_text_query(attribute_type, entity_type):
    return attribute_type == "Generic string" and entity_type == "Dimension"

query_11 = "Search for all products containing 'wireless'."
attribute_type_11 = "Generic string"  # Text-based search
entity_type_11 = "Dimension"
print(is_text_query(attribute_type_11, entity_type_11))  # Output: True

ATTRIBUTE_TYPES = {
    "Continuous": ["sales", "temperature", "revenue", "age", "growth", "score", "price"],
    "Categorical": ["region", "category", "department", "store"],
    "Ordered": ["rank", "order", "level"],
    "Boolean": ["true", "false"],
    "Database ID": ["id", "identifier", "user_id", "product_id"],
    "Generic string": ["name", "description", "keyword", "phrase"]
}

ENTITY_TYPES = {
    "Fact": ["sales", "temperature", "revenue", "growth", "score", "price"],
    "Dimension": ["region", "category", "department", "store", "user", "product"]
}
"""